In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aims-data-set/test_set.csv
/kaggle/input/aims-data-set/train_set.csv
/kaggle/input/aims-data-set/sample submission file.csv


In [2]:
pip install catboost


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


In [4]:
train = pd.read_csv("/kaggle/input/aims-data-set/train_set.csv")
test  = pd.read_csv("/kaggle/input/aims-data-set/test_set.csv")
sub   = pd.read_csv("/kaggle/input/aims-data-set/sample submission file.csv")
train.head(6)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,TransactionID,Date,Time,CustomerID,Age,Gender,Location,StoreID,OnlineOrderFlag,ProductID,...,PaymentType,PromotionApplied,DiscountAmount,GiftWrap,ShippingMethod,DeliveryTime,Weather,Event,CustomerSatisfaction,ReturnFlag
0,9255,11/16/2021,22:54:01,308,31,Female,City_13,41.0,False,688,...,Credit Card,True,0.000000,False,NaN,NaN,Rainy,Christmas Market,3,True
1,1562,12/11/2023,12:50:45,168,30,Other,City_12,NaN,True,291,...,Online Payment,False,0.000000,False,Overnight,1.0,Snowy,NaN,4,False
2,1671,12/15/2020,4:07:21,188,43,Other,City_8,5.0,False,931,...,Debit Card,False,11.522384,False,NaN,NaN,Snowy,Black Friday,4,True
3,6088,12/6/2020,5:40:48,327,27,Male,City_13,NaN,True,862,...,Credit Card,False,2.472306,False,Express,1.0,Rainy,Black Friday,3,True
4,6670,12/6/2022,1:15:52,22,70,Male,City_10,NaN,True,249,...,Cash,True,0.000000,True,Express,1.0,Sunny,NaN,1,True
5,5934,11/4/2020,17:42:06,440,28,Female,City_2,NaN,True,717,...,Cash,True,19.172805,True,Express,3.0,Snowy,Black Friday,4,False


In [5]:
for df in [train, test]:
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['year'] = df['Date'].dt.year
        df['month'] = df['Date'].dt.month
        df['day'] = df['Date'].dt.day
        df['dow'] = df['Date'].dt.dayofweek
        df.drop('Date', axis=1, inplace=True)

    df['qty_discount'] = df['Quantity'] * df['DiscountAmount']
    df['promo_wrap']   = df['PromotionApplied'] * df['GiftWrap']


In [6]:
TARGET = "TotalPrice"

X = train.drop(TARGET, axis=1)
y = train[TARGET]

cat_cols = X.select_dtypes(include='object').columns.tolist()


In [7]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:

for col in cat_cols:
    x_train[col] = x_train[col].astype(str).fillna("missing")
    x_valid[col] = x_valid[col].astype(str).fillna("missing")
    test[col]    = test[col].astype(str).fillna("missing")


In [9]:
model = CatBoostRegressor(iterations=3600,learning_rate=0.045,depth=9,l2_leaf_reg=6,min_data_in_leaf=25,loss_function='RMSE',eval_metric='RMSE',random_seed=42,verbose=300)          
 
model.fit(
    x_train, y_train,
    cat_features=cat_cols,
    eval_set=(x_valid, y_valid),
    use_best_model=True)


0:	learn: 111.9251095	test: 114.5314114	best: 114.5314114 (0)	total: 106ms	remaining: 6m 19s
300:	learn: 60.9826792	test: 63.5044507	best: 63.4858551 (295)	total: 8.32s	remaining: 1m 31s
600:	learn: 51.1649581	test: 63.5460389	best: 63.3816307 (360)	total: 18.6s	remaining: 1m 32s
900:	learn: 42.2227982	test: 63.5711427	best: 63.3816307 (360)	total: 28.9s	remaining: 1m 26s
1200:	learn: 34.7967737	test: 63.9210682	best: 63.3816307 (360)	total: 39.2s	remaining: 1m 18s
1500:	learn: 28.8168164	test: 64.0551160	best: 63.3816307 (360)	total: 49.4s	remaining: 1m 9s
1800:	learn: 24.2710148	test: 64.1920907	best: 63.3816307 (360)	total: 59.7s	remaining: 59.6s
2100:	learn: 20.7048221	test: 64.2512207	best: 63.3816307 (360)	total: 1m 10s	remaining: 50.1s
2400:	learn: 17.6581686	test: 64.3608670	best: 63.3816307 (360)	total: 1m 20s	remaining: 40.2s
2700:	learn: 15.0904977	test: 64.4353643	best: 63.3816307 (360)	total: 1m 30s	remaining: 30.1s
3000:	learn: 13.1555479	test: 64.4550927	best: 63.3816307

In [10]:
preds = model.predict(x_valid)
rmse = mean_squared_error(y_valid, preds, squared=False)
print( rmse)


63.38163059290452


In [11]:
sub[TARGET] = model.predict(test)
sub.to_csv("submission.csv", index=False)



In [12]:
from IPython.display import FileLink

FileLink("submission.csv")



/kaggle/working/submission.csv